In [ ]:
!pip install faiss-cpu      #for installation of faiss

from sentence_transformers import SentenceTransformer
import json
import faiss
import numpy as np
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Build FAISS Index in rag_pipeline
def build_faiss_index(chunks_path):
    # Load model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Load chunks from file
    chunks = []
    with open(chunks_path, "r", encoding="utf-8") as f:
        for line in f:
            chunks.append(json.loads(line))

    # Create FAISS index
    embedding_dim = len(chunks[0]['embedding'])
    index = faiss.IndexFlatL2(embedding_dim)
    embeddings = np.array([chunk['embedding'] for chunk in chunks]).astype('float32')
    index.add(embeddings)

    return model, index, chunks


In [ ]:
# 2. Retrieve function
def retrieve_chunks(query, model, index, chunks, top_k=5):
    query_embedding = model.encode(query).astype('float32')
    D, I = index.search(np.array([query_embedding]), top_k)
    results = [chunks[i] for i in I[0]]
    return results

In [ ]:
# 3. Load generator model
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    device=-1  # CPU; use 0 for GPU
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# 4. Answer generation
def generate_answer(query, model, index, chunks, top_k=5):

    # Step 1: Retrieve
    retrieved = retrieve_chunks(query, model, index, chunks, top_k=top_k)

    # Step 2: Prepare context
    context = "\n\n".join([f"Source: {r.get('source', 'Unknown')}\n{r['text']}" for r in retrieved])

    # Step 3: Build prompt
    prompt = f"""
    You are an expert eligibility officer.
    Using only the context below, answer the question truthfully.
    If the answer is not in the context, say "I cannot find relevant information."

    Context:
    {context}

    Question: {query}
    Answer:
    """

    # Step 4: Generate
    output = generator(prompt, max_new_tokens=200)

    # Step 5: Collect citations
    #citations = list({r.get('source', 'Unknown') for r in retrieved})
    citations = [f"{r.get('source', 'Unknown')} — {r['text']}" for r in retrieved]

    return output[0]["generated_text"], citations

In [ ]:
# 5. Run everything
chunks_path = r"/content/drive/MyDrive/australia_student_visa_chunks_embeddings.jsonl"
model, index, chunks = build_faiss_index(chunks_path)


# Query-1
query = "Is the student eligible for Australia Student Visa?"
answer, citations = generate_answer(query, model, index, chunks, top_k=5)

print("\nFinal Answer:\n", answer)
print("\nCitations:\n")
for c in citations:
    print("- " + c)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2576 > 512). Running this sequence through the model will result in indexing errors



Final Answer:
 Yes

Citations:

- Student and Child Student — 157A (Design date 11/13) - Page 3
© COMMONWEALTH OF AUSTRALIA, 2013
Student visa application checklist
Specific application checklists providing detailed information 
on student visa requirements based on your individual 
circumstances are available at  
www.immi.gov.au/students/checklists/
Qualifying visas for student visa applications lodged in 
Australia
If you are applying in Australia for a student visa you must 
either be the current holder of a qualifying visa or have held a 
certain visa no more than 28 days prior to making your 
application. More information on qualifying visas is available 
from the department’s website  
www.immi.gov.au/students/students/qualifying-visas-
for-student-visa-applications-lodged-australia.htm
Students and the skilled migration program
Australia has a well-deserved reputation for high-quality 
education and training. We continue to welcome overseas 
students, and appreciate the contri

In [ ]:
chunks_path = r"/content/drive/MyDrive/australia_student_visa_chunks_embeddings.jsonl"
model, index, chunks = build_faiss_index(chunks_path)

# Query-2
query = "What are the eligibility requirements for a Australia Student Visa?"
answer, citations = generate_answer(query, model, index, chunks, top_k=5)

print("\nFinal Answer:\n", answer)
print("\nCitations:\n")
for c in citations:
    print("- " + c)


Final Answer:
 You must either be the current holder of a qualifying visa or have held a certain visa no more than 28 days prior to making your application.

Citations:

- Student and Child Student — 157A (Design date 11/13) - Page 3
© COMMONWEALTH OF AUSTRALIA, 2013
Student visa application checklist
Specific application checklists providing detailed information 
on student visa requirements based on your individual 
circumstances are available at  
www.immi.gov.au/students/checklists/
Qualifying visas for student visa applications lodged in 
Australia
If you are applying in Australia for a student visa you must 
either be the current holder of a qualifying visa or have held a 
certain visa no more than 28 days prior to making your 
application. More information on qualifying visas is available 
from the department’s website  
www.immi.gov.au/students/students/qualifying-visas-
for-student-visa-applications-lodged-australia.htm
Students and the skilled migration program
Australia has

In [ ]:
chunks_path = r"/content/drive/MyDrive/australia_student_visa_chunks_embeddings.jsonl"
model, index, chunks = build_faiss_index(chunks_path)

# Query-3
query = "Do I need to provide proof of funds for an Australian Student Visa?"
answer, citations = generate_answer(query, model, index, chunks, top_k=5)

print("\nFinal Answer:\n", answer)
print("\nCitations:\n")
for c in citations:
    print("- " + c)



Final Answer:
 Yes

Citations:

- Student and Child Student — 157A (Design date 11/13) - Page 3
© COMMONWEALTH OF AUSTRALIA, 2013
Student visa application checklist
Specific application checklists providing detailed information 
on student visa requirements based on your individual 
circumstances are available at  
www.immi.gov.au/students/checklists/
Qualifying visas for student visa applications lodged in 
Australia
If you are applying in Australia for a student visa you must 
either be the current holder of a qualifying visa or have held a 
certain visa no more than 28 days prior to making your 
application. More information on qualifying visas is available 
from the department’s website  
www.immi.gov.au/students/students/qualifying-visas-
for-student-visa-applications-lodged-australia.htm
Students and the skilled migration program
Australia has a well-deserved reputation for high-quality 
education and training. We continue to welcome overseas 
students, and appreciate the contri

In [ ]:
chunks_path = r"/content/drive/MyDrive/australia_student_visa_chunks_embeddings.jsonl"
model, index, chunks = build_faiss_index(chunks_path)

# Query-4
query = "Is health insurance mandatory for an Australian Student Visa?"
answer, citations = generate_answer(query, model, index, chunks, top_k=5)

print("\nFinal Answer:\n", answer)
print("\nCitations:\n")
for c in citations:
    print("- " + c)



Final Answer:
 Yes

Citations:

- Student and Child Student — hat you and any family members 
accompanying you must maintain adequate arrangements for 
health insurance while in Australia. Further information can be 
found in the information sheet Applying for a student visa, 
available from the department’s website  
www.immi.gov.au/allforms/pdf/applying-student.pdf
Length of student visa
Provided that you have adequate arrangements for health 
insurance for the relevant period, the maximum duration your 
student visa can be granted is outlined below.
For courses other than Postgraduate Research (subclass 574) 
visa the maximum is:
• up to March 15 of the following year if your course is longer 
than 10 months and finishes at the end of the Australian 
academic year (November – December);
• up to 2 months longer than the duration of your course if 
the course is longer than 10 months and finishes between 
January and October;
• up to one month longer than the duration of your course 